In [4]:
from selenium.webdriver.common.keys import Keys
import scrape_functions as kzd
import sys
import calendar
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException, StaleElementReferenceException
import re
from selenium.webdriver.support.ui import Select
import random
import time
from selenium import webdriver
import os
import numpy as np
import importlib
importlib.reload(sys.modules['scrape_functions'])
from datetime import date

In [23]:
# Starting code
geko_path = "/Users/giovannachaves/Documents/BSE/Intro to Text Mining/HW3/Text-Mining-HW3.geckodriver.exe"

destination='/Users/giovannachaves/Documents/BSE/Intro to Text Mining/HW3/Text-Mining-HW3'
link = 'https://www.booking.com/'

profile, browser, download_path = kzd.start_up(destination, link, download=False)

In [24]:
# Clicking on the different buttoms to input things:

# Click on  place
place = input('Where do you want to go?')
search1 = browser.find_element('xpath','//input[@placeholder="Where are you going?"]')
search1.send_keys(place)

# Clik on dates:
kzd.check_and_click(
    browser, '//button[@class="d47738b911 e246f833f7 fe211c0731"]', type='xpath')

time.sleep(3)

dates=browser.find_elements('xpath',
    '//table[@class="aadb8ed6d3"]/tbody/tr/td/span')
    
from_day = input('Day from which you want to look for')
if int(from_day)< date.today().day:
    print('Wrong from_day')

to_day = input('Day until which you want to look for')


for date1 in dates:
    if date1.get_attribute("data-date") == f"2023-02-{from_day}":
        # date1.click() this is the old webdriver version
        browser.execute_script("arguments[0].click();", date1)
    if date1.get_attribute("data-date") == f"2023-02-{to_day}":
        # date1.click()
        browser.execute_script("arguments[0].click();", date1)
        break
        
kzd.check_and_click(browser, 'button.d4b6b7a9e7', type='css')
time.sleep(2)

Where do you want to go?Valencia
Day from which you want to look for02
Day until which you want to look for04


In [25]:
# LOOPING OVER WEBPAGES:

def get_number_pages(browser):
    '''
    Get the number of pages. 
    '''
    a = browser.find_elements('xpath',
        '//button[text() and @class="fc63351294 f9c5690c58"]')
    total_pages = int(a[-1].text)
    return(total_pages)

In [30]:
total_pages = get_number_pages(browser)
print(f"Total pages: {total_pages}")

ratings_list=[]
hotels_list=[]
prices_list=[]

for current_page in range(total_pages):    
    hotels = browser.find_elements('xpath','//div[@class="fcab3ed991 a23c043802"]')
    ratings = browser.find_elements('xpath','//div[@class="b5cd09854e d10a6220b4"]')
    prices = browser.find_elements('xpath','//span[@data-testid="price-and-discounted-price"]')
    
    for i in ratings:
        ratings_list.append(i.text)
        
    for i in hotels:
        hotels_list.append(i.text)

    for i in prices:
        prices_list.append(i.text)

    next_page_btn = browser.find_element('xpath', '//button[contains(@aria-label, "Next page")]')
    next_page_btn.click()
    time.sleep(4)

Total pages: 17


In [31]:
ratings_list

['8.6',
 '8.3',
 '9.0',
 '8.0',
 '6.9',
 '9.1',
 '8.8',
 '8.7',
 '8.1',
 '8.6',
 '8.3',
 '8.2',
 '8.2',
 '7.8',
 '8.6',
 '8.3',
 '9.2',
 '8.4',
 '8.7',
 '7.5',
 '7.4',
 '8.6',
 '8.6',
 '8.9',
 '8.4',
 '8.4',
 '8.3',
 '7.8',
 '8.2',
 '9.0',
 '9.2',
 '8.0',
 '7.9',
 '8.6',
 '7.7',
 '7.6',
 '8.8',
 '8.5',
 '9.4',
 '8.5',
 '8.9',
 '9.2',
 '7.9',
 '8.4',
 '7.7',
 '9.0',
 '7.9',
 '8.2',
 '7.9',
 '7.9',
 '8.4',
 '9.4',
 '8.6',
 '8.4',
 '9.0',
 '8.1',
 '8.4',
 '7.9',
 '7.7',
 '8.6',
 '8.2',
 '8.5',
 '9.0',
 '8.8',
 '9.1',
 '8.2',
 '8.3',
 '9.4',
 '7.7',
 '8.8',
 '8.2',
 '8.8',
 '7.7',
 '8.0',
 '8.6',
 '8.7',
 '7.5',
 '7.3',
 '8.4',
 '8.4',
 '8.2',
 '8.1',
 '7.8',
 '9.0',
 '8.1',
 '8.4',
 '7.9',
 '8.6',
 '8.6',
 '8.3',
 '7.9',
 '9.0',
 '8.5',
 '9.3',
 '8.0',
 '8.4',
 '8.9',
 '8.5',
 '8.7',
 '8.9',
 '8.3',
 '8.2',
 '8.8',
 '8.0',
 '8.4',
 '8.9',
 '7.1',
 '9.1',
 '8.5',
 '8.0',
 '8.6',
 '8.4',
 '6.0',
 '8.8',
 '8.2',
 '9.5',
 '8.6',
 '10',
 '9.1',
 '9.4',
 '8.5',
 '8.7',
 '8.7',
 '9.5',
 '7.9',
 

In [32]:
hotels_list

['Vincci Lys',
 'Bet Apartments - Apartments Catedral',
 'City Center Atico 360',
 'Loft independiente con Piscina compartida en Centro de Valencia',
 'PLAZA REDONDA brugada home',
 'Macflats Ayuntamiento',
 'Hotel RH Sorolla Centro',
 'Hotel Mediterraneo Valencia',
 'Welcomer Apartments Valencia',
 'The Little Corner B&B',
 'One Shot Colón 46',
 'One Shot Palacio Reina Victoria 04',
 'SH Valencia Palace',
 'Holiday Inn Express Ciudad de las Ciencias, an IHG Hotel',
 'Vincci Mercat',
 'Sercotel Sorolla Palace',
 'Only YOU Hotel Valencia',
 'Casa del Patriarca',
 'Hotel Malcom and Barret',
 'Resa Patacona',
 'Hospederia del Pilar',
 'Hotel Boutique Balandret',
 'Zenit Valencia',
 'Soho Boutique Turia',
 'Catalonia Excelsior',
 'Catalonia Excelsior',
 'Ad Hoc Monumental 1881',
 'Eurostars Rey Don Jaime',
 'Silken Puerta Valencia',
 'Hotel Plaza Mercado & Spa',
 'Helen Berger Boutique Hotel',
 'Hotel Medium Valencia',
 'Dormavalencia Hostel Regne',
 'Melia Plaza Valencia',
 'Sol Playa',
 

In [33]:
prices_list

['€ 278',
 '€ 283',
 '€ 384',
 '€ 213',
 '€ 116',
 '€ 164',
 '€ 139',
 '€ 240',
 '€ 187',
 '€ 130',
 '€ 174',
 '€ 218',
 '€ 220',
 '€ 164',
 '€ 242',
 '€ 158',
 '€ 350',
 '€ 147',
 '€ 154',
 '€ 160',
 '€ 113',
 '€ 206',
 '€ 208',
 '€ 205',
 '€ 210',
 '€ 210',
 '€ 188',
 '€ 198',
 '€ 182',
 '€ 368',
 '€ 255',
 '€ 185',
 '€ 125',
 '€ 238',
 '€ 128',
 '€ 177',
 '€ 164',
 '€ 195',
 '€ 180',
 '€ 186',
 '€ 167',
 '€ 239',
 '€ 217',
 '€ 319',
 '€ 152',
 '€ 191',
 '€ 194',
 '€ 207',
 '€ 155',
 '€ 203',
 '€ 274',
 '€ 198',
 '€ 385',
 '€ 161',
 '€ 243',
 '€ 208',
 '€ 132',
 '€ 129',
 '€ 120',
 '€ 230',
 '€ 245',
 '€ 257',
 '€ 204',
 '€ 150',
 '€ 230',
 '€ 195',
 '€ 168',
 '€ 215',
 '€ 160',
 '€ 166',
 '€ 221',
 '€ 174',
 '€ 95',
 '€ 176',
 '€ 351',
 '€ 204',
 '€ 140',
 '€ 130',
 '€ 231',
 '€ 248',
 '€ 151',
 '€ 224',
 '€ 204',
 '€ 175',
 '€ 144',
 '€ 298',
 '€ 180',
 '€ 178',
 '€ 156',
 '€ 156',
 '€ 178',
 '€ 238',
 '€ 209',
 '€ 445',
 '€ 143',
 '€ 187',
 '€ 265',
 '€ 167',
 '€ 122',
 '€ 380',
 